In [1]:
#IMPORTAR LIBRERIAS A USAR

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df=pd.read_csv('DataCleaned.csv')
df=df.drop(['Unnamed: 0'], axis=1)
df.columns

Index(['trip_id', 'duration', 'start_time', 'end_time', 'start_lat',
       'start_lon', 'end_lat', 'end_lon', 'bike_id', 'plan_duration',
       'trip_route_category', 'passholder_type', 'start_station',
       'end_station', 'Date_start_time', 'Hour_start_time',
       'Minute_start_time', 'Date_end_time', 'Hour_end_time',
       'Minute_end_time'],
      dtype='object')

In [2]:
#GENERAR OTRA VARIABLE PARA LA BASE DE DATOS
df_model = df[ ['passholder_type','duration','trip_route_category','start_station','end_station','Hour_start_time','Hour_end_time'] ]

In [ ]:
#SUSTITUIR LOS TIPOS DE PASE POR VALORES NUMERICOS PARA ENTRENAR EL MODELO
df_model['passholder_type']=df_model['passholder_type'].replace({'Annual Pass': 1})
df_model['passholder_type']=df_model['passholder_type'].replace({'Flex Pass': 2})
df_model['passholder_type']=df_model['passholder_type'].replace({'Monthly Pass': 3})
df_model['passholder_type']=df_model['passholder_type'].replace({'One Day Pass': 4})
df_model['passholder_type']=df_model['passholder_type'].replace({'Testing': 5})
df_model['passholder_type']=df_model['passholder_type'].replace({'Walk-up': 6})

In [4]:
#OMITIR LOS VALORES NAN DE LA VARIABLE df_model
df_model = df_model.dropna()
print(df_model['passholder_type'].unique())

[6. 3. 4. 1. 2. 5.]


In [5]:
#Resetear los indices
df_model=df_model.reset_index()
df_model=df_model.drop(['index'], axis=1)

In [6]:
#IMPORTAR LIBRERIAS PARA CONSTRUIR LOS MODELOS
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


#DEFINIR LAS VARIABLES x e y (tanto de entrenamiento como prueba) EN BASE A LA BASE DE DATOS
#x SON TODAS LAS VARIABLES A EXCPECION DE LA COLUMNA de 'passholder_type'
#y CORRESPONDE A LA COLUMNA 'passholder_type'
#EL SET DE DATOS SE DIVIDIO EN 80% PARA ENTRENAMIENTO Y EL 20% PARA PRUEBA

X=df_model.drop('passholder_type',axis=1)
X = pd.get_dummies(X)
X['trip_route_category_One Way'] = X['trip_route_category_One Way'].astype(float)
X['trip_route_category_Round Trip'] = X['trip_route_category_Round Trip'].astype(float)
y=df_model.passholder_type

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#CONSTRUCCION MODELO DE LINEAR REGRESSION

from sklearn.linear_model import LinearRegression, Lasso


lm=LinearRegression()
lm.fit(X_train,y_train)
np.mean(cross_val_score(lm,X_train,y_train, scoring='neg_mean_absolute_error', cv=3))

-1.1664485366256407

In [ ]:
#CONSTRUCCION MODELO DE LASSO REGRESSION
lm_l=Lasso() 
lm_l.fit(X_train,y_train)
np.mean(cross_val_score(lm_l,X_train,y_train, scoring='neg_mean_absolute_error', cv=3))

-1.2247615878609308

In [7]:
#CONSTRUCCION MODELO DE RANDOM FOREST CLASSIFICATION
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
np.mean(cross_val_score(rf,X_train,y_train, scoring='neg_mean_absolute_error', cv=3))

-0.8100031366223058

In [ ]:
#PREDECIR LOS RESULTADOS USANDO EL SET DE PRUEBA
pred_lm=lm.predict(X_test)
pred_lml=lm_l.predict(X_test)
pred_rf=rf.predict(X_test)

In [ ]:
#CALCULAR LOS ERRORES
from sklearn.metrics import mean_absolute_error
er_lm=mean_absolute_error(y_test,pred_lm)
er_lml=mean_absolute_error(y_test,pred_lml)
er_rf=mean_absolute_error(y_test,pred_rf)

In [ ]:
print(er_lm)
print(er_lml)
print(er_rf)

1.1669531305909169
1.223995500318972
0.8018382300225119


In [ ]:
#LEER Y USAR EL ARCHIVO 'test_set.csv' PARA PREDECIR EL TIPO DE PASE PARA CADA CASO
#DE IGUAL MANERA OMITIR TODOS LOS VALORES NAN y VALORES NO NUMERICOS EN LA COLUMNA 'bike_id'
df_test=pd.read_csv('test_set.csv')
df_test = df_test.dropna()
df_test=df_test[pd.to_numeric(df_test['bike_id'], errors='coerce').notnull()]
df_test

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,trip_id,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,trip_route_category,start_station,end_station
0,17059130,12,1/1/2017 0:24,1/1/2017 0:36,34.058319,-118.246094,34.058319,-118.246094,6351,Round Trip,3028,3028
1,17059129,17,1/1/2017 0:28,1/1/2017 0:45,34.049980,-118.247162,34.043732,-118.260139,5836,One Way,3027,3018
2,17059126,20,1/1/2017 0:39,1/1/2017 0:59,34.063389,-118.236160,34.044159,-118.251579,6529,One Way,3066,3055
3,17059125,12,1/1/2017 0:43,1/1/2017 0:55,34.048851,-118.246422,34.050140,-118.233238,6029,One Way,3029,3079
4,17061379,48,1/1/2017 0:56,1/1/2017 1:44,34.049198,-118.252831,34.049198,-118.252831,6680,Round Trip,3063,3063
...,...,...,...,...,...,...,...,...,...,...,...,...
569881,179408277,19,12/31/2021 23:29,12/31/2021 23:48,34.040989,-118.255798,34.041130,-118.267982,14027,One Way,3033,3011
569882,179408276,8,12/31/2021 23:40,12/31/2021 23:48,34.044701,-118.252441,34.051941,-118.243530,20006,One Way,3031,3030
569883,179409877,43,12/31/2021 23:47,1/1/2022 0:30,34.044701,-118.252441,34.044701,-118.252441,16214,Round Trip,3031,3031
569884,179409876,42,12/31/2021 23:48,1/1/2022 0:30,34.044701,-118.252441,34.044701,-118.252441,13336,Round Trip,3031,3031


In [ ]:
#CONVERIR LA COLUMNA 'bike_id' A VALORES NUMERICOS
df_test['bike_id'] = pd.to_numeric(df_test['bike_id'])

In [ ]:
#RESETEAR LOS INDICES
df_test=df_test.reset_index()
df_test=df_test.drop(['index'], axis=1)

In [ ]:
#AGREGAR NUEVAS COLUMNAS SEPARANDO FECHAS Y HORAS EN DIFERENTES COLUMNAS

df_test['start_time'] = pd.to_datetime(df_test['start_time'])
df_test['Date_start_time'] = pd.to_datetime(df_test['start_time']).dt.strftime('%d/%m/%Y')
df_test['Hour_start_time'] = df_test['start_time'].dt.hour
df_test['Minute_start_time'] = pd.to_datetime(df_test['start_time']).dt.minute

df_test['end_time'] = pd.to_datetime(df_test['end_time'])
df_test['Date_end_time'] = pd.to_datetime(df_test['end_time']).dt.strftime('%d/%m/%Y')
df_test['Hour_end_time'] = df_test['end_time'].dt.hour
df_test['Minute_end_time'] = pd.to_datetime(df_test['end_time']).dt.minute

In [ ]:
#IMPRIMIR COLUMNAS DE LA BASE DE DATOS
df_test.columns

Index(['trip_id', 'duration', 'start_time', 'end_time', 'start_lat',
       'start_lon', 'end_lat', 'end_lon', 'bike_id', 'trip_route_category',
       'start_station', 'end_station', 'Date_start_time', 'Hour_start_time',
       'Minute_start_time', 'Date_end_time', 'Hour_end_time',
       'Minute_end_time'],
      dtype='object')

In [ ]:
#GENERAR UNA BASE DE DATOS PARA SER USADA POR EL MODELO SIGUIENDO LA MISMA NOMENCLATURA COMO SE MUESTRA ABAJO
df_testM = df_test[ ['duration','trip_route_category','start_station','end_station','Hour_start_time','Hour_end_time'] ]

In [ ]:
#SEPARAR LOS DATOS DE 'trip_route_category' EN DOS COLUMNAS, CORRESPNDIENTES A 'One Way' Y 'Round Trip'
df_testM= pd.get_dummies(df_testM)
df_testM

,duration,start_station,end_station,Hour_start_time,Hour_end_time,trip_route_category_One Way,trip_route_category_Round Trip
0,12,3028,3028,0,0,0,1
1,17,3027,3018,0,0,1,0
2,20,3066,3055,0,0,1,0
3,12,3029,3079,0,0,1,0
4,48,3063,3063,0,1,0,1
...,...,...,...,...,...,...,...
552403,19,3033,3011,23,23,1,0
552404,8,3031,3030,23,23,1,0
552405,43,3031,3031,23,0,0,1
552406,42,3031,3031,23,0,0,1


In [ ]:
#PREDECIR LOS RSULTADOS USANDO EL MODELO DE RANDOM FOREST QUE FUE EL ALGORITMO CON MEJOR DESEMPEÑO
pred_Fin=rf.predict(df_testM)

In [ ]:
#GENERAR DOS VARIABLES: 
#UNA CONTIENE LA COLUMNA 'trip_id' 
#Y LA OTRA LOS RESULTADOS PREDECIDOS POR EL MODELO SEGUN EL TIPO DE 'passholder_type'
rcol1=df_test['trip_id']
rcol2 = pd.DataFrame(pred_Fin, columns = ['passholder_type'])
rcol2=round(rcol2)

In [ ]:
#VISUALIZAR EL NUMERO DE VECES QUE EL MODELO PREDIJO CADA CATEGORIA DONDE SI RECORDAMOS CADA VALOR NUMERICO 
#DE LA IZQUIERDA REPRESENTA EL TIPO DE PASE DE LA SIGUEINTE MANERA
#'Annual Pass'= 1
#'Flex Pass'= 2
#'Monthly Pass'= 3
#'One Day Pass'= 4
#'Testing'= 5
#'Walk-up'= 6

#POR LO QUE DE LA TABLA SE PUEDE CONCLUIR QUE EL TIPO DE PASE CON MAS DEMANDA ES EL 'Monthly_Pass' Y EL MENOS 
#DEMANDADO 'Testing'
#SIENDO SIMILAR A CUANDO HICIMOS EL ANALISIS DE DATOS DONDE SE OBSERVA EL MISMO COMPORTAMIENTO O TENDENCIA

sta2 = rcol2.pivot_table(columns=['passholder_type'], aggfunc='size')
sta2

passholder_type
1.0     14336
2.0      3993
3.0    360058
4.0     17147
5.0         4
6.0    156870
dtype: int64

In [ ]:
#CONCATENAR LAS VARIABLES rcol1 y rcol2 CORRESPONDENTES a 'trip_id' y 'passholder_type'
Results=pd.concat([rcol1, rcol2], axis=1, ignore_index=True)

In [ ]:
#NOMBRAR LAS COLUMNAS DE LA NUEVA VARIABLE
Results.columns = ['trip_id', 'passholder_type']

Results.head(3)

,trip_id,passholder_type
0,17059130,3.0
1,17059129,6.0
2,17059126,3.0


In [ ]:
#SUSTITUIR LOS VALORES NUMERICOS POR LA NOMENCLATURA INICIAL CORRESPONDIENTE A CADA TIPO DE PASE
Results['passholder_type']=Results['passholder_type'].replace({1 : 'Annual Pass'})
Results['passholder_type']=Results['passholder_type'].replace({2 : 'Flex Pass'})
Results['passholder_type']=Results['passholder_type'].replace({3 : 'Monthly Pass'})
Results['passholder_type']=Results['passholder_type'].replace({4 : 'One Day Pass'})
Results['passholder_type']=Results['passholder_type'].replace({5 : 'Testing'})
Results['passholder_type']=Results['passholder_type'].replace({6 : 'Walk-up'})

In [ ]:
Results.head(3)

,trip_id,passholder_type
0,17059130,Monthly Pass
1,17059129,Walk-up
2,17059126,Monthly Pass


In [ ]:
#GUARDAR LOS RESULTADOS EN EL ARCHIVO 'Submission.csv'
Results.to_csv("Submission.csv",index=False) 

In [54]:
#GUARDAR MODELO RANDOM FOREST
import pickle
import logging

print("Mode score:",rf.score(X_train,y_train))


Mode score: 0.9392032979343101


In [25]:
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(rf, file)